In [1]:
import pandas as pd
import numpy as np

import string
import datetime
import email
import re
from tqdm import tqdm
import logging
import nltk
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip uninstall tensorflow

In [2]:
# Read the CSV File
raw_emails_df = pd.read_csv('giskard_dataset.csv', sep=';')

In [3]:
def get_text_from_email(msg):
    '''
    To get the content from email objects
    '''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

In [4]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, raw_emails_df['Message']))
raw_emails_df.drop('Message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    raw_emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
raw_emails_df['Content'] = list(map(get_text_from_email, messages))

In [5]:
raw_emails_df

,Unnamed: 0,Target,Message-ID,Date,From,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Content
0,0,internal company policy,<32715630.1075840547840.JavaMail.evans@thyme>,"Wed, 11 Jul 2001 08:29:27 -0700 (PDT)",legalonline-compliance@enron.com,Confidential Information and Securities Trading,1.0,text/plain; charset=us-ascii,7bit,Office of the Chairman - Enron Wholesale Servi...,"GILBERT-SMITH@mailman.enron.com, Gilbert-smith...",,,"\ExMerge - Gilbert-smith, Doug\need to read",GILBERTSMITH-D,doug gilbert smith 6-25-02.PST,"To:GILBERT-SMITH, DOUGLAS\nEmail:doug.gilbert-..."
1,1,alliances / partnerships,<8721012.1075863428380.JavaMail.evans@thyme>,"Tue, 26 Jun 2001 09:40:17 -0700 (PDT)",j.kaminski@enron.com,"FW: Venezuela FX and inflation forecasts, June...",1.0,text/plain; charset=us-ascii,7bit,"Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...",'vkaminski@aol.com',,,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",Kaminski-V,VKAMINS (Non-Privileged).pst,\n\n -----Original Message-----\nFrom: \tKoepk...
2,2,internal company operations,<8687721.1075852656109.JavaMail.evans@thyme>,"Wed, 13 Jun 2001 06:05:07 -0700 (PDT)",steven.kean@enron.com,Enron Expatriates in India,1.0,text/plain; charset=us-ascii,7bit,"Kean, Steven </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...","Lay, Kenneth </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",,,\JSKILLIN (Non-Privileged)\Deleted Items,Skilling-J,JSKILLIN (Non-Privileged).pst,"John Brindle, David Cromley and others in the ..."
3,3,internal company operations,<16562450.1075846168630.JavaMail.evans@thyme>,"Mon, 21 Aug 2000 00:30:00 -0700 (PDT)",steven.kean@enron.com,Re: PLEASE READ & RESPOND,1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,Steven J Kean,Michael Terraso,,,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,Yes. Contact either Gia or John and get the b...
4,4,internal projects -- progress and strategy,<955111.1075858690252.JavaMail.evans@thyme>,"Sat, 16 Jun 2001 17:36:06 -0700 (PDT)",b..sanders@enron.com,FW: ISO's Response to BPA Rebuttal of Sheffrin...,1.0,text/plain; charset=us-ascii,7bit,"Sanders, Richard B. </O=ENRON/OU=NA/CN=RECIPIE...","'sbishop@gibbs-bruns.com', 'jalexander@gibbs-b...",,,"\Sanders, Richard B (Non-Privileged)\Sanders, ...",Sanders-R,"Sanders, Richard B (Non-Privileged).pst",\nThis is something interesting to put in your...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1616,internal company operations,<18983060.1075847582386.JavaMail.evans@thyme>,"Sun, 13 May 2001 12:37:00 -0700 (PDT)",steven.kean@enron.com,Message from Clark C. Smith,1.0,text/plain; charset=us-ascii,7bit,Steven J Kean,Maureen McVicker,,,\Steven_Kean_June2001_1\Notes Folders\All docu...,KEAN-S,skean.nsf,include on the distribution list and send out ...
875,1637,alliances / partnerships,<29879754.1075863427653.JavaMail.evans@thyme>,"Mon, 25 Jun 2001 11:33:50 -0700 (PDT)",j.kaminski@enron.com,RE:,1.0,text/plain; charset=us-ascii,7bit,"Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...","'""Aruna Inalsingh@Reval"" <Aruna_Inalsingh@Reva...","Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...",,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",Kaminski-V,VKAMINS (Non-Privileged).pst,"Aruna,\n\nI shall be in London this week. Plea..."
876,1646,talking points,<3001077.1075863428054.JavaMail.evans@thyme>,"Tue, 26 Jun 2001 08:58:12 -0700 (PDT)",j.kaminski@enron.com,RE: Marketing Plan & Meetings,1.0,text/plain; charset=us-ascii,7bit,"Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...","Kohli, Sandeep </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",Kaminski-V,VKAMINS (Non-Privileged).pst,"Sandeep,\n\nThis looks great. I would talk to ..."
877,1670,internal projects -- progress and strategy,<13669071.1075863428696.JavaMail.evans@thyme>,"Wed, 27 Jun 2001 03:14:41 -0700 (PDT)",j.kaminski@enron.com,RE: Test Message,1.0,text/plain; charset=us-ascii,7bit,"Kaminski, Vince J </O=

In [6]:
emails_df = raw_emails_df[['X-Folder','Subject','Content', 'Date','Target']]

In [7]:
emails_df = emails_df.rename(columns={'X-Folder': 'Folder'})

In [8]:
emails_df

,Folder,Subject,Content,Date,Target
0,"\ExMerge - Gilbert-smith, Doug\need to read",Confidential Information and Securities Trading,"To:GILBERT-SMITH, DOUGLAS\nEmail:doug.gilbert-...","Wed, 11 Jul 2001 08:29:27 -0700 (PDT)",internal company policy
1,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...","FW: Venezuela FX and inflation forecasts, June...",\n\n -----Original Message-----\nFrom: \tKoepk...,"Tue, 26 Jun 2001 09:40:17 -0700 (PDT)",alliances / partnerships
2,\JSKILLIN (Non-Privileged)\Deleted Items,Enron Expatriates in India,"John Brindle, David Cromley and others in the ...","Wed, 13 Jun 2001 06:05:07 -0700 (PDT)",internal company operations
3,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,Re: PLEASE READ & RESPOND,Yes. Contact either Gia or John and get the b...,"Mon, 21 Aug 2000 00:30:00 -0700 (PDT)",internal company operations
4,"\Sanders, Richard B (Non-Privileged)\Sanders, ...",FW: ISO's Response to BPA Rebuttal of Sheffrin...,\nThis is something interesting to put in your...,"Sat, 16 Jun 2001 17:36:06 -0700 (PDT)",internal projects -- progress and strategy
...,...,...,...,...,...
874,\Steven_Kean_June2001_1\Notes Folders\All docu...,Message from Clark C. Smith,include on the distribution list and send out ...,"Sun, 13 May 2001 12:37:00 -0700 (PDT)",internal company operations
875,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",RE:,"Aruna,\n\nI shall be in London this week. Plea...","Mon, 25 Jun 2001 11:33:50 -0700 (PDT)",alliances / partnerships
876,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",RE: Marketing Plan & Meetings,"Sandeep,\n\nThis looks great. I would talk to ...","Tue, 26 Jun 2001 08:58:12 -0700 (PDT)",talking points
877,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Se...",RE: Test Message,"Mike,\n\nThanks for your message. The meeting ...","Wed, 27 Jun 2001 03:14:41 -0700 (PDT)",internal projects -- progress and strategy


In [9]:
# To Confirm there are no blank data 
emails_df = emails_df.drop(emails_df.query('Date == "" | Subject == "" | Content == "" | Folder ==""').index)
emails_df.reset_index(drop=True, inplace=True) # To reset the index of the dropped rows
    

In [23]:
def clean_data(columns):
    """
    To Clean Data from columns of dataframes
    Removes unnecessary character, lowercases words 
    and performs stemming/lemitizing
    """
    columns = re.sub("=\n", "", columns) # To remove the '=' at every EOL
    #columns = re.sub(r"&", "", columns) # To remove the '&' in company names
    stopwords = nltk.corpus.stopwords.words('english')
    newstopwords = ['ect', 'hou', 'com', 'recipient', 'cc', 'na', 'ees', 'th', 'pm', 'pg']
    stopwords.extend(newstopwords)
    #data = re.sub(r"\S*https?:\S*", "", columns)
    #data = re.sub('[^a-zA-Z]', ' ', columns) # removes all characters except a-z and A-Z
    data = columns.lower() 
    data = nltk.word_tokenize(data)
    data = [word for word in data if not word in (stopwords)]
    
    #     if clean == 'stem':
    #         ps = PorterStemmer() 
    #         data = [ps.stem(word) for word in data if not word in (stopwords)] # stemming all words that are not stopwords 
    #     elif clean == 'lem':
    #         lm = WordNetLemmatizer() # initialize lemmatizing 
    #         data = [lm.lemmatize(word) for word in data if not word in (stopwords)] # lemmitizing all words that are not stopwords 

    #     else:
    #         data = [word for word in data if not word in (stopwords)] # stemming all words that are not stopwords 


    data = ' '.join(data)
        #data = ' '.join(i for i in data if i not in (string.punctuation))
        
    return data

In [10]:
emails_df['Texttransformed'] = emails_df['Date'] + emails_df['Folder'] + " " + emails_df['Subject'] + " " + emails_df['Content']


In [25]:
emails_df['Texttransformed'] = emails_df.apply(lambda x: clean_data(x['Texttransformed']), axis=1)


In [11]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
tokenized = emails_df['Texttransformed'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


In [13]:
tokenized

0      [101, 21981, 1010, 2340, 21650, 2541, 5511, 10...
1      [101, 10722, 2063, 1010, 2656, 12022, 2541, 56...
2      [101, 21981, 1010, 2410, 12022, 2541, 5757, 10...
3      [101, 12256, 1010, 2538, 15476, 2456, 4002, 10...
4      [101, 2938, 1010, 2385, 12022, 2541, 2459, 102...
                             ...                        
846    [101, 3103, 1010, 2410, 2089, 2541, 2260, 1024...
847    [101, 12256, 1010, 2423, 12022, 2541, 2340, 10...
848    [101, 10722, 2063, 1010, 2656, 12022, 2541, 55...
849    [101, 21981, 1010, 2676, 12022, 2541, 6021, 10...
850    [101, 21981, 1010, 2423, 21650, 2541, 2410, 10...
Name: Texttransformed, Length: 851, dtype: object

In [14]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [15]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(851, 54597)

In [16]:
#x = torch.tensor(train).to(torch.int64)
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 142731408384 bytes. Buy new RAM!
